<a href="https://colab.research.google.com/github/lamld203844/kapur-and-otsu-segmentation/blob/main/gradio_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s et

In [2]:
# Install watermark
!pip install -q watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.9 MB/s eta 0:00:00


In [3]:
# Load watermark
%load_ext watermark

# Display the version of Python and Gradio
%watermark -v -p gradio,cv2,numpy,matplotlib,PIL,scipy,skimage

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

gradio    : 4.15.0
cv2       : 4.8.0
numpy     : 1.23.5
matplotlib: 3.7.1
PIL       : 9.4.0
scipy     : 1.11.4
skimage   : 0.19.3



In [4]:
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

from scipy.ndimage import median_filter
from skimage.filters import threshold_multiotsu
from skimage.filters import threshold_otsu

%matplotlib inline

In [5]:
def basic_preprocessing(input_image, kernel_size):
    """
    Preprocessing: average, median filter, and histogram equalization with size and
    shape of the mask should be selected.((kernel_size = 3 by default))
    """
    # Convert the input image to grayscale
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # average filter
    blurred_image = cv2.blur(gray_image, (kernel_size, kernel_size))

    # median filter
    median_filtered_image = median_filter(blurred_image, size=kernel_size)

    # histogram equalizing
    hist_equ_img = cv2.equalizeHist(gray_image)


    return blurred_image, median_filtered_image, hist_equ_img

In [6]:
def calculate_psnr(input_image):
    # Convert the input image to grayscale
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Calculate MSE
    mse = np.mean((gray_image.astype(np.float64) / 255) ** 2)
    if mse == 0:
        return "Same Image"

    # Calculate PSNR
    PIXEL_MAX = 255.0
    psnr = 20 * np.log10(PIXEL_MAX) - 10 * np.log10(mse)

    return psnr

In [7]:
def kapur_segment(input_image):
    # Convert the input image to grayscale
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Normalize to 0 - 255
    image = cv2.normalize(gray_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

    # Apply Kapur's method
    thresholds = threshold_multiotsu(image)

    # Apply the thresholds to get a segmented image
    segmented_image = np.digitize(image, bins=thresholds)

    return segmented_image

In [8]:
def otsu_segment(input_image):
    # Convert the input image to grayscale
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)

    # Normalize to 0 - 255
    image = cv2.normalize(gray_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

    # Apply Otsu's thresholding
    thresh = threshold_otsu(image)
    binary = image > thresh

    return binary

In [9]:
def calculate_metrics(pred, mask):
    '''
    Input: image and corresponding mask after normalization [0, 255] -> [0, 1], for example
    Output: metrics
    '''

    # normalize eh_mask [0, 255] -> [0, 1]
    norm_mask = np.where(mask, 1, 0)
    # convert 3 channel to 1 channel
    norm_mask_uint8 = norm_mask.astype(np.uint8)
    mask = cv2.cvtColor(norm_mask_uint8, cv2.COLOR_RGB2GRAY)

    # True positive
    tp = np.sum((pred == 1) & (mask == 1))

    # True negative
    tn = np.sum((pred == 0) & (mask == 0))

    # False positive
    fp = np.sum((pred == 1) & (mask == 0))

    # False negative
    fn = np.sum((pred == 0) & (mask == 1))

    # Sensitivity (also known as recall or true positive rate)
    sensitivity = tp / (tp + fn)

    # Specificity (also known as true negative rate)
    specificity = tn / (tn + fp)

    # Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)

    return sensitivity, specificity, accuracy

In [10]:
def main_function(img, kernel_sz, original_mask):

    # basic img information
    info = img.shape

    # basic preprocessing
    average_img, median_img, hist_equ_img = basic_preprocessing(img, kernel_sz)

    # psnr value
    psnr = calculate_psnr(img)

    # segmentation Kapur method
    kapur_seg_mask = kapur_segment(img)
    kapur_img = ((kapur_seg_mask / kapur_seg_mask.max()) * 255).astype(np.uint8) # Map the segmented image from 0-255
    # segmented_3_channel = cv2.cvtColor(segmented_image, cv2.COLOR_GRAY2BGR) # Convert the segmented image to a 3-channel image

    # segmentation otsu method
    otsu_seg_mask = otsu_segment(img)
    otsu_img = cv2.cvtColor((otsu_seg_mask * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR) # Convert binary mask to a 3-channel image

    # metrics calculation
    sens, spec, acc = calculate_metrics(otsu_seg_mask, original_mask)
    metric_overall = f'Otsu: {sens:.2f}, {spec:.2f}, {acc:.2f}'

    return info, average_img, median_img, hist_equ_img, psnr, kapur_img, otsu_img, metric_overall

In [11]:
inputs = [
    gr.Image(type='numpy', label="Input original image"),
    gr.Number(label='kernel size (average and median filter only)', value=3),
    gr.Image(type='numpy', label="Input original mask"),
]
outputs = [gr.Textbox(label="Shape"), # img info
            gr.Image(label="avg filtering"), # avg filtering
            gr.Image(label="median filtering"), # median filtering
            gr.Image(label="histogram equalized img"), # histogram equalizing
            gr.Number(label="psnr value"), # psnr value
            gr.Image(label="kapur_segmentation"), # segmentation Kapur method
            gr.Image(label="otsu_segmentation"), # segmentation Otsu method
           gr.Textbox(label="Metrics accuracy, specificity, sensitivity respectively"), # metrics
        ]

iface = gr.Interface(fn=main_function,
                     inputs=inputs,
                     outputs=outputs)

iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6c5d287c018ce10e73.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6c5d287c018ce10e73.gradio.live


In [12]:
iface.close()

Closing server running on port: 7860
